# CVA and DVA

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.optimize import minimize
from scipy.optimize import minimize_scalar
from math import exp
from math import log
from math import sqrt
from functools import partial
import locale

In [2]:
_ = locale.setlocale(locale.LC_ALL, '')

#### Exercise 20.13

In [3]:
T = 2.
K = 1500.
F0 = 1600.
risk_free_rate = .05 # continuous compounding
recovery_rate = .3
sigma_gold = .2      # Annual volatility of forward price of gold
default_year_offset = 1./24 # defaults take place in the middle of each month
q1 = .001667 # Monthly unconditional default probability in the first year
q2 = .0025   # Monthly unconditional default probability in the second year

In [4]:
# Possible times of default in the first and the second years of the forward contract
t1s = np.array([default_year_offset + i/12. for i in range(12)])
t2s = 1. + t1s

In [5]:
# Define d1, d2 as per the Black-Scholes-Merton formula for a futures or forward price
# Any of the arguments can be either a scalar or a numpy array
def  d1(S0, sigma, K, T, r=0.):
    return  (np.log(S0/K) + (r + np.square(sigma) / 2) * T) / (sigma * np.sqrt(T))
def  d2(S0, sigma, K, T, r=0.):
    return  d1(S0, sigma, K, T, r) - sigma * np.sqrt(T)

In [6]:
# Here we are valuing a call on a forward contract
def  ts_to_exposures(ts):
    return  exp(-risk_free_rate * T)\
        * (F0*norm.cdf(d1(F0, sigma_gold, K, ts)) - K*norm.cdf(d2(F0, sigma_gold, K, ts)))

In [7]:
v1s = ts_to_exposures(t1s)
v2s = ts_to_exposures(t2s)

In [8]:
cva = (1 - recovery_rate) * ((q1 * v1s).sum() + (q2 * v2s).sum())

In [9]:
print('CVA of the forward contract: {:s}'.format(locale.currency(cva, grouping=True)))

CVA of the forward contract: $5.73


#### Exercise 20.14

In [10]:
q_bank = 1e-3
recovery_rate_bank = .4

In [11]:
# Here we are valuing a put on a forward contract (as we are looking to the transaction
# through the eyes of the counterparty)
def  ts_to_exposures_dva(ts):
    return  exp(-risk_free_rate * T)\
        * (K*norm.cdf(-d2(F0, sigma_gold, K, ts)) - F0*norm.cdf(-d1(F0, sigma_gold, K, ts)))

In [12]:
v1s_cpy = ts_to_exposures_dva(t1s)
v2s_cpy = ts_to_exposures_dva(t2s)

In [13]:
dva = (1 - recovery_rate_bank) * (q_bank * np.concatenate((v1s_cpy, v2s_cpy))).sum()

In [14]:
print('DVA of the forward contract: {:s}'.format(locale.currency(dva, grouping=True)))

DVA of the forward contract: $0.97


#### Exercise 20.15

In [23]:
S0 = 52.
K = 50.
risk_free_rate = .05 # continuous compounding
sigma_stock = .3
T = 1
recovery_rate = 0.
q = .02    # Probability of default at maturity of option seller

**(a)** Pricing a non-dividend-paying Eurpoean call option.

In [16]:
c = S0*norm.cdf(d1(S0, sigma_stock, K, T, risk_free_rate))\
    - K*exp(-risk_free_rate*T)*norm.cdf(d2(S0, sigma_stock, K, T, risk_free_rate))

In [17]:
print('Value of the option assuming no possibility of default: {:s}'.format(locale.currency(c, grouping=True)))

Value of the option assuming no possibility of default: $8.41


**(b)** Pricing the same option with CVA.

In [18]:
cva = c * q
print('CVA: {:s}\nValue of the option with CVA: {:s}'.format(
    locale.currency(cva, grouping=True),
    locale.currency(c - cva, grouping=True)))

CVA: $0.17
Value of the option with CVA: $8.25


**(c)** Pricing the same option with delayed premium payment.

If the option's buyer pays the option price (with interest) at maturity, the exposure from default to the buyer will equal $max(S - K + c\cdot e^{r\cdot T}, 0)$. In other words, to find the new exposure we'll need to price an option on the same stock but with a different strike.

In [19]:
K_adjusted = K + c * exp(risk_free_rate * T)
c_adjusted = S0*norm.cdf(d1(S0, sigma_stock, K_adjusted, T, risk_free_rate))\
    - K_adjusted*exp(-risk_free_rate*T)*norm.cdf(d2(S0, sigma_stock, K_adjusted, T, risk_free_rate))
cva_adjusted = c_adjusted * q

In [26]:
print('Paying option price at maturity with interest leads to an adjusted CVA of: {:s}\n'
      'It thus reduces the cost of defaults by: {:s}, compared with paying the premium upfront'.format(
          locale.currency(cva_adjusted, grouping=True), 
          locale.currency(cva - cva_adjusted, grouping=True)))

Paying option price at maturity with interest leads to an adjusted CVA of: $0.09
It thus reduces the cost of defaults by: $0.08, compared with paying the premium upfront


**(d)** Pricing the DVA on the same option with delayed premium payment.

The exposure to the dealer will be $max(K + c\cdot e^{r\cdot T} - S, 0)$. In othwe words, to find the exposure to the dealer we'll need to price the corresponding European put option with a strike of $K + c\cdot e^{r\cdot T}$.

In [24]:
q_buyer = .01    # Probability of default at maturity of option buyer

In [25]:
p = K_adjusted*exp(-risk_free_rate*T)*norm.cdf(-d2(S0, sigma_stock, K_adjusted, T, risk_free_rate))\
    - S0*norm.cdf(-d1(S0, sigma_stock, K_adjusted, T, risk_free_rate))
dva = p * q_buyer

In [27]:
print('Paying option price at maturity with interest leads to a DVA of {:s} for the option\'s seller'.format(
          locale.currency(dva, grouping=True)))

Paying option price at maturity with interest leads to a DVA of $0.09 for the option's seller


#### Exercise 20.16

In [28]:
yield_spread = .0210
c = 4.1
T = 3.

In [29]:
print('Option price with CVA: {:s}'
      .format(locale.currency(c * exp(-yield_spread*T), grouping=True)))

Option price with CVA: $3.85
